In [1]:
from functools import partial

from astropaul.database import database_connection
import astropaul.html as html
import astropaul.lbt as lbt
import astropaul.phase as ph
import astropaul.targetlistcreator as tlc

%load_ext autoreload
%autoreload 2

import warnings

warnings.simplefilter('always')  # Show all warnings
warnings.showwarning = warnings._showwarnmsg_impl  # Use default formatting with stack trace


In [2]:
html_dir = "targets"
html.clear_directory(html_dir)

with database_connection() as conn:
    phase_event_defs = [
        ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
        ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
        ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
        ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
        ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
    ]

    name = "All Targets"
    creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
    creator.steps = [
        tlc.add_targets,
        # partial(tlc.filter_targets, criteria=lambda df: df["Target Name"] == "TIC 470710327"),
        # partial(tlc.filter_targets, criteria=lambda df: df["Target Name"] == "TIC 278352276"),
        # partial(tlc.filter_targets, criteria=lambda df: df["Target Name"].isin(["TIC 123098844", "TIC 278352276", "TIC 470710327"])),
        # partial(tlc.filter_targets, criteria=lambda df: df["Target Type"] == "RV Standard"),
        tlc.ancillary_data_from_tess,
        partial(tlc.add_columns_from_sql, table_name="RV Calibration Targets", primary_cols=["RV"]),
        tlc.add_tess_catalog_associations,
        tlc.add_lists,
        partial(tlc.add_database_table, table_name="Ephemerides"),
        tlc.tabulate_ephemerides,
        partial(tlc.add_database_table, table_name="DSSI Observations"),
        partial(tlc.add_database_table, table_name="HRCam Observations"),
        partial(tlc.add_database_table, table_name="Speckle Detections"),
        partial(tlc.add_database_table, table_name="PEPSI Observations"),
        tlc.add_pepsi_evaluations,
        tlc.add_rv_status,
        partial(tlc.add_member_phases, table_name="DSSI Observations", time_column="Mid JD"),
        # partial(
        #     tlc.add_system_configuration,
        #     table_name="DSSI Observations",
        #     time_column="Mid JD",
        #     eclipse_table="SIDE Observations",
        # ),
        partial(tlc.add_member_phases, table_name="PEPSI Observations", time_column="Mid JD"),
        partial(tlc.add_resource, resource="PEPSI SpectrumPlot", table_name="PEPSI Observations"),
        partial(tlc.add_database_table, table_name="ZorroAlopeke Observations"),
    ]
    tl = creator.calculate(verbose=False)
    html.render_observing_pages(tl, None, {}, html_dir)

tl.target_list

,Target Name,RA,Dec,RA HMS,Dec DMS,Target Type,Target Source,Vmag,Teff,PMra,...,List Speckle Resolved,List Table 1 Targets,List Table 2 Targets,List Zasche 2022,Num Ephemerides,Num DSSI Observations,Num HRCam Observations,Num Speckle Detections,Num PEPSI Observations,Num ZorroAlopeke Observations
0,HIP 142,0.453851,66.306080,00:01:48.92424,+66:18:21.888,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00...",7.3300,5660.00,6.271720,...,False,False,False,False,0.0,0,0,0,0,0
1,HIP 184,0.589740,11.006050,00:02:21.5376,+11:00:21.78,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00...",8.4800,5085.07,-4.947850,...,False,False,False,False,0.0,0,0,0,0,0
2,HIP 867,2.663211,12.820160,00:10:39.17064,+12:49:12.576,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00...",8.4700,6092.62,40.101800,...,False,False,False,False,0.0,0,0,0,0,0
3,HIP 870,2.669626,2.055630,00:10:40.71024,+02:03:20.268,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00...",7.5100,6426.41,-12.462200,...,False,False,False,False,0.0,0,0,0,0,0
4,HIP 920,2.851618,23.818360,00:11:24.38832,+23:49:06.096,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00...",8.2300,6141.55,-35.302300,...,False,False,False,False,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,TIC 70664819,87.276125,60.177269,05:49:06.27,+60:10:38.17,QuadEB,Zasche 2022 https://doi.org/10.1051/0004-6361/...,10.5480,8378.13,-0.100000,...,False,False,False,True,0.2,1,0,0,0,0
728,TIC 7215036,113.714542,-7.102839,07:34:51.49,-07:06:10.22,QuadEB,Zasche 2022 https://doi.org/10.1051/0004-6361/...,14.7330,7762.00,-1.005520,...,False,False,False,True,0.2,0,0,0,0,0
729,TIC 72555159,93.557708,-7.131711,06:14:13.85,-07:07:54.16,QuadEB,Zasche 2022 https://doi.org/10.1051/0004-6361/...,13.5430,7843.29,-0.018771,...,False,False,False,True,0.2,0,0,0,0,0
730,TIC 851090533,158.702292,-60.217750,10:34:48.55,-60:13:03.9,QuadEB,Zasche 2022 https://doi.org/10.1051/0004-6361/...,9.9312,NaN,-5.569330,...,False,False,False,True,0.2,0,0,0,0,0


In [4]:
tl.target_list.sort_values("Num HRCam Observations")


,Target Name,RA,Dec,RA HMS,Dec DMS,Target Type,Target Source,Vmag,Teff,PMra,...,List Resolved 2025-09-30,List Speckle Resolved,List Table 1 Targets,List Zasche 2022,Num Ephemerides,Num DSSI Observations,Num HRCam Observations,Num Speckle Detections,Num PEPSI Observations,Num ZorroAlopeke Observations
0,HIP 142,0.453851,66.306080,00:01:48.92424,+66:18:21.888,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00...",7.330,5660.0,6.271720,...,False,False,False,False,0.0,0,0,0,0,0
494,TIC 78333248,88.730875,26.354792,05:54:55.41,+26:21:17.25,QuadEB,Kostov 2023 doi.org/10.1093/mnras/stad2947,13.942,7566.0,0.682764,...,False,False,False,False,3.0,0,0,0,0,0
495,TIC 80893927,91.577000,28.671039,06:06:18.48,+28:40:15.74,QuadEB,Kostov 2023 doi.org/10.1093/mnras/stad2947,12.502,6196.3,5.158730,...,False,False,False,False,2.0,0,0,0,0,0
496,TIC 91600053,88.366583,16.891742,05:53:27.98,+16:53:30.27,QuadEB,Kostov 2023 doi.org/10.1093/mnras/stad2947,13.000,6539.0,0.601975,...,False,False,False,False,4.0,0,0,0,0,0
497,TIC 93373156,86.750375,-20.903569,05:47:00.09,-20:54:12.85,QuadEB,Kostov 2023 doi.org/10.1093/mnras/stad2947,14.687,5749.0,1.243630,...,False,False,False,False,2.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,TIC 387096013,140.130583,-54.410581,09:20:31.34,-54:24:38.09,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,12.262,NaN,-4.948610,...,False,False,False,True,2.1,0,2,2,0,0
420,TIC 300446218,111.594917,-66.281067,07:26:22.78,-66:16:51.84,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,15.010,NaN,-2.855720,...,True,False,True,False,4.0,0,2,2,0,1
435,TIC 348651800,121.724333,-12.435678,08:06:53.84,-12:26:08.44,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,11.405,NaN,-29.824000,...,True,True,True,False,4.0,2,2,6,4,1
416,TIC 285655079,233.992500,-57.122536,15:35:58.2,-57:07:21.13,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,13.573,8463.0,-2.426600,...,False,False,False,False,4.0,0,2,2,0,0
